# Statement

La rapidez del compresor es de 1 500 rpm. El compresor tiene un diámetro de cilindro del cilindro de 3.125 in, una carrera de 3.26 in, y
una razón biela a cigüeñal de 3.5. La presión de admisión es la atmosférica (14.7 psia), la presión pico del cilindro es de 132 psig y la presión efectiva media (mep) es de 26 psig. El flujo es de 8.9 pies cúbicos
por minuto a la presión efectiva media, lo cual da 1.6 hp.


El diámetro interior del compresor es de 3.125 in. La fuerza dinámica
que actúa sobre la cabeza varía de 0 a 1 000 lb cada ciclo, a partir
de 130 psi de presión en el cilindro. Un empaque no confinado de
asbesto-cobre de 0.06 in de espesor cubre totalmente el punto de contacto cabeza-cilindro. El espesor de la cabeza en los puntos de sujeción (exclusivas con aletas de enfriamiento) es de 0.4 in.

![](./figura_d_2.png)

In [11]:
import numpy as np
import pandas as pd
from io import StringIO
from utils import *
from sympy import Eq, solve, Symbol, symbols

In [12]:
data ='''
d cuerdas dr at cuerdas dr at
1 0.0730 64 0.0527 72 0.0550 0.0028
2 0.0860 56 0.0628 64 0.0657 0.0039
3 0.0990 48 0.0719 56 0.0758 0.0052
4 0.1120 40 0.0795 48 0.0849 0.0066
5 0.1250 40 0.0925 44 0.0955 0.0083
6 0.1380 32 0.0974 40 0.1055 0.0101
8 0.1640 32 0.1234 36 0.1279 0.0147
10 0.1900 24 0.1359 32 0.1494 0.0200
12 0.2160 24 0.1619 28 0.1696 0.0258
1/4 0.2500 20 0.1850 28 0.2036 0.0364
5/16 0.3125 18 0.2403 24 0.2584 0.0581
3/8 0.3750 16 0.2938 24 0.3209 0.0878
7/16 0.4375 14 0.3447 20 0.3725 0.1187
1/2 0.5000 13 0.4001 20 0.4350 0.1600
9/16 0.5625 12 0.4542 18 0.4903 0.2030
5/8 0.6250 11 0.5069 18 0.5528 0.2560
3/4 0.7500 10 0.6201 16 0.6688 0.3730
7/8 0.8750 0.7307 14 0.7822 0.5095
1 1.0000 0.8376 12 0.8917 0.6630
1 1/8 1.1250 0.9394 12 1.0167 0.8557
1 1/4 1.2500 1.0644 12 1.1417 1.0729
1 3/8 1.3750 1.1585 12 1.2667 1.3147
1 1/2 1.5000 1.2835 12 1.3917 1.5810
'''
data = pd.read_csv(StringIO(data), sep=' ', skipinitialspace=True)
data.head()
data.to_csv('tornillos_unc.csv', index=False, )
extra_at = '''
1.8995
2.4982
3.2477
3.9988
4.9340
5.9674
7.0989
8.3286
9.6565
11.0826
'''

# extra_at = pd.read_csv(StringIO(extra_at), sep=' ', skipinitialspace=True, header=None)
# extra_at.columns = ['at']

# data['at'] = np.nan
# data['at'].iloc[:len(extra_at)] = extra_at['at']

# data.to_csv('tornillos_unc.csv', index=False, )

In [13]:
# Asuma un perno de 5/16-18 UNC-2A
diameter_perno = 5/16 # in
pitch = 18 # hilos por pulgada
diameter_low = 0.2403 # in
area_tension = 0.0524 # in^2

# Asuma un material SAE grado 7 
SP = 105e3 # psi
SY = 115e3 # psi
SU = 133e3 # psi

# Asuma una pre-cara de 80% de la resistencia de prueba
porcentaje_pre_carga = 0.7
pre_carga = porcentaje_pre_carga * SP * area_tension # lb
print('pre_carga =', pre_carga, 'lb')

# Modulo de young de los materiales
E_acero = 30e6 # psi
E_aluminio = 10.4e6 # psi

# Longitud de junta
l_c = 2 * diameter_perno + 1/4 # in
print('l_c =', l_c, 'in')

# Asuma que la longitud del perno es 1.25 in
l_perno = 1.25  # in
l_junta = 1.25 # in

l_s = l_perno - l_c
l_t = l_junta - l_s
print('l_s =', l_s, 'in')
print('l_t =', l_t, 'in')

j = diameter_perno / l_junta # factor de junta
print('j =', j)

pre_carga = 3851.4 lb
l_c = 0.875 in
l_s = 0.375 in
l_t = 0.875 in
j = 0.25


In [14]:
table_11_8 = pd.read_csv('table_11_8.csv')
table_11_8.head()

,j,p0,p1,p2,p3
0,0.1,0.4389,-0.9197,0.8901,-0.3187
1,0.2,0.6118,-1.1715,1.0875,-0.3806
2,0.3,0.6932,-1.2426,1.1177,-0.3845
3,0.4,0.7351,-1.2612,1.1111,-0.3779
4,0.5,0.7580,-1.2632,1.0979,-0.3708


In [15]:
def get_row_values(df:pd.DataFrame, j:float):
    
    row = df.loc[df['j'] == j]
    if len(row) > 0:
        return row.iloc[0][1:].to_numpy()
    
    upper_values = df.loc[df['j'] > j].iloc[0] [1:].to_numpy()
    lower_values = df.loc[df['j'] < j].iloc[-1] [1:].to_numpy()

    upper_j = df.loc[df['j'] > j].iloc[0] [:1].to_numpy()
    lower_j = df.loc[df['j'] < j].iloc[-1] [:1].to_numpy()

    return (j - lower_j) / (upper_j - lower_j) * (upper_values - lower_values) + lower_values


In [16]:
p_0, p_1, p_2, p_3 = get_row_values(table_11_8, j)
print('p_0 =', p_0, 'p_1 =', p_1, 'p_2 =', p_2, 'p_3 =', p_3)

ratio_module = E_aluminio / E_acero
print('ratio_module =', ratio_module)

c_t = calculate_c_r(ratio_module, p_0, p_1, p_2, p_3)
print('c_t =', c_t)

p_0 = 0.6525000000000001 p_1 = -1.20705 p_2 = 1.1025999999999998 p_3 = -0.38255
ratio_module = 0.3466666666666667
c_t = 0.35062636136296305


In [18]:
area_perno = np.pi * pow(diameter_perno, 2) / 4
print('area_perno =', area_perno, 'in^2')

k_b_prima = calculate_k_b_prima(j, area_tension, area_perno, l_s=l_s, l_t=l_t)
print('k_b_prima =', f'{k_b_prima:,.4g}')

k_m = Symbol('k_m')
eqt = Eq(c_t, (k_b_prima) / (k_m + k_b_prima) )
k_m = solve(eqt)[0]
print('k_m =', f'{k_m:,.4g}')

area_perno = 0.07669903939428206 in^2
k_b_prima = 1.112e+06
k_m = 2.059e+6
